In [1]:
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session
from dotenv import load_dotenv
import os

load_dotenv()


connection_parameters = {
   "account": os.getenv('account_snow'),
   "user": os.getenv('user_snow'),
   "password":os.getenv('password_snow'),
   "role": "ACCOUNTADMIN",
   "database": "NEW_DB", 
   "schema": "PUBLIC"
  
}  

session = Session.builder.configs(connection_parameters).create()

In [2]:
employee_data = [
    [1, 4],
    [2,5],
    [3,6]
]

employee_schema = ["A", "B",]

df_emp = session.createDataFrame(employee_data, schema=employee_schema)
df_emp.show()

-------------
|"A"  |"B"  |
-------------
|1    |4    |
|2    |5    |
|3    |6    |
-------------



In [3]:
from snowflake.snowpark.functions import udf, col

from snowflake.snowpark.types import IntegerType

@udf(name='a_plus_b', input_types=[IntegerType(), IntegerType()], return_type=IntegerType(), is_permanent=False, replace=True)

def a_plus_b(a: int, b: int) -> int:

    return a+b

df_emp.withColumn('A_PLUS_B', a_plus_b(col('A'), col('B')))

In [5]:
df_emp.show()

-------------
|"A"  |"B"  |
-------------
|1    |4    |
|2    |5    |
|3    |6    |
-------------



In [6]:
import pandas as pd
from snowflake.snowpark.functions import pandas_udf
from snowflake.snowpark.types import IntegerType, PandasSeriesType

@pandas_udf(name='multiplier', input_types=[PandasSeriesType(IntegerType()), PandasSeriesType(IntegerType())], return_type=PandasSeriesType(IntegerType()), is_permanent=False, replace=True)
def multiplier(column1: pd.Series, column2: pd.Series) -> pd.Series:
    return column1 * column2


df_emp.withColumn('MULTIPLIED', multiplier(col('A'), col('B'))).show()

----------------------------
|"A"  |"B"  |"MULTIPLIED"  |
----------------------------
|1    |4    |4             |
|3    |6    |18            |
|2    |5    |10            |
----------------------------

